In [ ]:
!pip install -U openai

import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [3]:
from openai import OpenAI
client = OpenAI()

prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

カレーのレシピをご紹介します。家庭で簡単に作れる基本的なカレーのレシピです。

### 材料（4人分）

- 鶏肉（または牛肉、豚肉） 300g
- 玉ねぎ 1個
- にんじん 1本
- じゃがいも 2個
- カレールウ 1箱（約180g）
- 水 600ml
- サラダ油 大さじ2
- 塩 適量
- 胡椒 適量
- ガーリックパウダー（オプション） 小さじ1
- お好みでトッピング（ゆで卵、パセリ、福神漬けなど）

### 作り方

1. **材料の下ごしらえ**
   - 鶏肉は一口大に切ります。
   - 玉ねぎは薄切りにします。
   - にんじんは輪切りまたはいちょう切りにします。
   - じゃがいもは皮をむいて一口大に切ります。

2. **炒める**
   - 大きめの鍋にサラダ油を熱し、中火で玉ねぎを炒めます。しんなりするまで、約5分程度炒めます。
   - 次に鶏肉を加え、表面が白くなるまで炒めます。
   - にんじんとじゃがいもも加え、更に数分間炒めます。

3. **煮込む**
   - 鍋に水を加え、煮立たせます。沸騰したら、アクを取り除きます。
   - 蓋をして、中火で20分程度煮込みます。野菜が柔らかくなるまで加熱します。

4. **カレーの素を加える**
   - 鍋の火を弱め、カレールウを少しずつ加え、よく混ぜます。完全に溶けるまでかき混ぜ続けます。
   - お好みで塩や胡椒、ガーリックパウダーで味を調えます。

5. **仕上げ**
   - カレーがとろっとしてきたら火を止め、そのまま数分間置きます。時間が経つとさらに味がなじみます。

6. **盛り付け**
   - ご飯を皿に盛り、その上にカレーをかけます。お好みでゆで卵やパセリ、福神漬けをトッピングして完成です。

### おすすめの食べ方
カレーはライスだけでなく、ナンやパンとも相性が良いです。また、カレーうどんやカレードリアとしてアレンジして楽しむのもおすすめです。

ぜひお試しください！


In [4]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)

```
{
  "材料": ["米", "カレールウ", "玉ねぎ", "にんじん", "じゃがいも", "鶏肉", "水", "油", "塩"],
  "手順": [
    "米を洗い、炊飯器で炊く。",
    "玉ねぎ、にんじん、じゃがいもを皮をむき、適当な大きさに切る。",
    "鶏肉も一口大に切る。",
    "フライパンに油を熱し、玉ねぎが透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、さらに炒める。",
    "鍋に水を加え、野菜と肉を煮立たせる。",
    "アクを取り、蓋をして中火で15分程煮る。",
    "火を止め、カレールウを入れて混ぜ、完全に溶かす。",
    "再度火にかけ、約5分程煮込んで味をなじませる。",
    "ご飯を盛り付け、カレーをかけて完成。"
  ]
}
```


In [6]:
from pydantic import BaseModel

system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。
"""

class Recipe(BaseModel):
    materials: list[str]
    steps: list[str]

response = client.responses.parse(
    model="gpt-5",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
    text_format=Recipe,
)
print(response.output_parsed)

materials=['ご飯 4皿分', '豚こま切れまたは鶏もも肉 350g', '玉ねぎ 2個（約400g・薄切り）', 'にんじん 1本（乱切り）', 'じゃがいも 2個（大きめの一口大）', 'サラダ油 大さじ1', 'おろしにんにく 小さじ1（又は1片）', 'おろししょうが 小さじ1', '水 750ml', 'カレールウ（中辛） 1/2箱（約100g）', 'ローリエ 1枚（あれば）', '塩・こしょう 少々', '薄力粉 大さじ1（肉用・任意）', 'すりおろしりんご 1/4個分（任意）', 'ウスターソース 大さじ1（任意）', 'はちみつ 小さじ1（任意）'] steps=['下ごしらえ：玉ねぎは薄切り、にんじんは乱切り、じゃがいもは大きめの一口大に切り水に10分さらす。肉は一口大に切り、塩こしょうで下味。好みで薄力粉を薄くまぶす。', '鍋に油を中火で熱し、にんにく・しょうがを香りが出るまで炒める。玉ねぎを加え、弱めの中火で10〜15分、しんなり甘みが出るまで炒める。', '鍋の中央を空けて肉を入れ、色が変わるまで炒める。にんじん・水気を切ったじゃがいもを加え、2〜3分炒め合わせる。', '水とローリエを入れて強火で沸かし、アクを丁寧に取る。ふたを少しずらして弱火〜中弱火で約15分、具材が柔らかくなるまで煮る。', '火を止めてローリエを取り出し、カレールウを割り入れて溶かす。すりおろしりんご、ウスター、はちみつを加える（任意）。', '弱火でとろみと照りが出るまで5〜10分ほど煮込み、焦げないよう時々混ぜる。味を見て塩・こしょうで調える。濃ければ水を少量、緩ければ弱火でさらに煮て調整。', '火を止めて10分ほど休ませると味がなじむ。再度温め、ご飯にかけて盛り付ける。好みで福神漬け・らっきょうを添える。']
